In [5]:
import requests
import json
import pandas as pd
from config import API_KEY

In [6]:
HEADERS = {'Authorization': f'Bearer {API_KEY}'}
SEARCH_URL = 'https://api.yelp.com/v3/businesses/search'

In [7]:
def fetch_restaurants(location):
    params = {
        'term': 'restaurants',
        'location': location,
        'limit': 50,
        'radius': 8000  # in meters
    }
    response = requests.get(SEARCH_URL, headers=HEADERS, params=params)
    return response.json()['businesses']

In [8]:
def process_businesses(businesses):
    names = []
    types = []
    addresses = []
    distances = []
    lats = []
    longs = []
    ratings = []
    prices = []

    for business in businesses:
        try:
            name = business["name"]
            type = business['categories'][0]['title'] if business['categories'] else "N/A"
            address = ', '.join(business['location']['display_address'])
            distance = business.get("distance", "N/A")
            lat = business["coordinates"]["latitude"]
            long = business["coordinates"]["longitude"]
            rating = business.get("rating", "N/A")
            price = business.get("price", "N/A")
        except (KeyError, IndexError):
            type = "N/A"

        names.append(name)
        types.append(type)
        addresses.append(address)
        distances.append(distance)
        lats.append(lat)
        longs.append(long)
        ratings.append(rating)
        prices.append(price)

    return pd.DataFrame({
        'Name': names,
        'Type': types,
        'Address': addresses,
        'Distance(Meters)': distances,
        'Latitude': lats,
        'Longitude': longs,
        'Rating': ratings,
        'Price': prices
    })

In [9]:
honolulu_businesses = fetch_restaurants('Honolulu, HI')
honolulu_df = process_businesses(honolulu_businesses)

In [10]:
honolulu_df.to_csv('honolulu_restaurants.csv', index=False)


In [11]:
honolulu_df.head()

,Name,Type,Address,Distance(Meters),Latitude,Longitude,Rating,Price
0,Monkeypod Kitchen,Tacos,"2169 Kālia Rd, Unit 111, Honolulu, HI 96815",3987.963043,21.278553,-157.832735,4.3,$$
1,Olili,Desserts,"2380 Kuhio Ave, Fl 2, Urban Honolulu, HI 96815",4367.977031,21.278242,-157.824512,4.7,N/A
2,Oli's Kitchen,Diners,"1009 University Ave, Ste MP2, Honolulu, HI 96826",3357.794621,21.292112,-157.821350,4.7,$$
3,Basalt,New American,"2255 Kuhio Ave, Honolulu, HI 96815",4104.829150,21.279612,-157.826944,4.3,$$
4,Duke's Waikiki,Hawaiian,"2335 Kalakaua Ave, Ste 116, Honolulu, HI 96815",4339.092605,21.276917,-157.827891,4.0,$$
